## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-08-14-50-45 +0000,nyt,Oct. 7 Anniversary Arrives During Heated Final...,https://www.nytimes.com/2025/10/07/nyregion/ma...
1,2025-10-08-14-50-42 +0000,nypost,Mom who appeared on paternity TV show allegedl...,https://nypost.com/2025/10/08/us-news/mom-alle...
2,2025-10-08-14-50-25 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/08/us/jam...
3,2025-10-08-14-47-41 +0000,wapo,"Trump threatened shutdown layoffs. So far, he ...",https://www.washingtonpost.com/business/2025/1...
4,2025-10-08-14-47-00 +0000,wsj,James Comey Pleads Not Guilty to Charges of Ly...,https://www.wsj.com/us-news/james-comey-federa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,39
57,new,22
238,gaza,15
25,shutdown,13
1031,pay,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
309,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,92
5,2025-10-08-14-46-38 +0000,nyt,Trump Says Chicago Mayor and Illinois Governor...,https://www.nytimes.com/2025/10/08/us/trump-pr...,92
10,2025-10-08-14-43-00 +0000,wsj,President Trump called for the jailing of Chic...,https://www.wsj.com/politics/policy/trump-call...,90
317,2025-10-07-17-41-13 +0000,nypost,Trump says Dems ‘have no leader’ in shutdown s...,https://nypost.com/2025/10/07/us-news/trump-ri...,88
178,2025-10-08-03-26-15 +0000,nyt,Trump Suggests Furloughed Federal Workers May ...,https://www.nytimes.com/2025/10/07/us/politics...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
309,92,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
347,59,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
8,58,2025-10-08-14-44-00 +0000,wsj,"In Japan, as in the U.S., a new leader wants t...",https://www.wsj.com/economy/central-banking/go...
5,53,2025-10-08-14-46-38 +0000,nyt,Trump Says Chicago Mayor and Illinois Governor...,https://www.nytimes.com/2025/10/08/us/trump-pr...
44,47,2025-10-08-13-33-06 +0000,nypost,Ex-FBI Director James Comey insists he’s innoc...,https://nypost.com/2025/10/08/us-news/ex-fbi-d...
274,46,2025-10-07-20-45-00 +0000,wsj,White House Says Federal Workers’ Back Pay Dur...,https://www.wsj.com/politics/policy/customurl-...
181,39,2025-10-08-02-46-58 +0000,nyt,Israel Marks Two-Year Anniversary of Oct. 7 in...,https://www.nytimes.com/2025/10/07/world/middl...
298,36,2025-10-07-19-20-11 +0000,nypost,"Tesla unveils long-awaited cheaper Model Y, Mo...",https://nypost.com/2025/10/07/business/tesla-u...
301,35,2025-10-07-18-50-35 +0000,cbc,Gold prices hit record high as anxious investo...,https://www.cbc.ca/news/business/gold-prices-r...
200,35,2025-10-08-00-44-44 +0000,nyt,Athens Democracy Forum: As Democracy Erodes Gl...,https://www.nytimes.com/2025/10/07/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
